# Resumo do código

### Código que gera o ficheiro: 2_Delta_2022Id
---
O objectivo é receber dados dos ficheiros totais da Delta e devolver um ficheiro com os dados dos produtos e das lojas relevantes para o estudo da evolução do padrão de vendas (dias homólogos do ano anterior). 

Devolve um ficheiro que pode entrar no código 2 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados da Delta Todos__: Lojas e produtos todos
> - Todas as métricas possíveis de retirar só dos dados Delta


- Outputs

> **2_Delta_2022Id**:
> - Ficheiro com todas as métricas nas datas homólogas correspondentes aos dias em que ocorreram as missões


In [4]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)

# Ler ficheiro com tudo e os específicos

- Tudo

In [2]:
%%time
dfTudo=pd.read_csv('D:\\B&N Dados\\Delta\\Stocks2022\\StocksDelta2022.csv')

CPU times: total: 11.6 s
Wall time: 17.6 s


> Passar para datetime

In [5]:
dfTudo['DATA']= pd.to_datetime(dfTudo['DATA'], format='%Y-%m-%d')

# Encontrar os nomes no dataFrame de tudo

In [36]:
#dfTudo['DESC_ARTIGO'].sort_values().unique()


#Lojas:
                  #Reposição=Sim
#CNT AMADORA
#CNT BRAGA
#CNT CASCAIS
#CNT COIMBRASHOPPING
#CNT MATOSINHOS
                  #Reposição=Não
#CNT LEIRIA
#CNT SANTAREM
#CNT VIANA
#MDL T. VEDRAS                                              
#CNT PORTIMÃO

#CAFÉ DELTA Q QALIDUS 80CAP
#CAFÉ DELTA Q MYTHIQ 80CAP
#CAFÉ DELTA Q QALIDUS 10CAP
#CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G
#CAFÉ BELLISSIMO INTENSO 200GR
#CAFÉ DELTA RITUAL MU 220G
#CAFÉ SOLÚVEL DELTA FRASCO 200G
#CAFÉ SOLÚVEL DELTA FRASCO 100G
#CEVADA SOLÚVEL DELTA FRASCO 200G

#Produto novo

array(['100% SALICÓRNIA MOIDA 45G', 'AZEITONA GALEGA QAMPO FR 210G',
       'AZEITONA OXIDADA QAMPO FR 210G',
       'BARRA CROFFEE CAF AMEND. CHOCO 3*30G',
       'BARRA CROFFEE CAF CARAM SALG 5*22G',
       'BARRA CROFFEE CAF. AVEIA CAN AME 5*22G',
       'BEBIDA CEREAIS DELTA C/20%CAFE FR 200G',
       'BEBIDA CEREAIS DELTA CEREAIS+FIBRA 200G',
       'CACAU EM PÓ DELTA 5*25 GR', 'CAFE BELLISSIMO OR 200G',
       'CAFFELATTE GO CHILL DELTA 230 ML', 'CAFÉ ANGOLA DELTA 16 PAST',
       'CAFÉ BARCO DESCAFÍINADO SACO 500G', 'CAFÉ BARCO GRÃO SACO 500G',
       'CAFÉ BARCO MOÍDO 100% CX 250G', 'CAFÉ BELLISSIMO DECAF10 CAPS',
       'CAFÉ BELLISSIMO INTENSO 10 CAP', 'CAFÉ BELLISSIMO INTENSO 200GR',
       'CAFÉ BELLISSIMO ORIGINALE 10 CAP',
       'CAFÉ DELTA BIO MOAGEM UNIVERSAL 220G',
       'CAFÉ DELTA CEVADA CHICÓRIA CENTEIO 200G',
       'CAFÉ DELTA CHICÓRIA + SOLÚVEL LATA 250',
       'CAFÉ DELTA CHÁVENA ÚNICA 2*250G',
       'CAFÉ DELTA COLÔMBIA GRÃO 220 G',
       'CAFÉ DELTA DESC 

# Fazer subsets da dataFrame total

> - Por loja e produto

In [6]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\lojas.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()

#produtos=["CAFÉ DELTA Q QALIDUS 80CAP","CAFÉ DELTA Q MYTHIQ 80CAP","CAFÉ DELTA Q QALIDUS 10CAP","CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G","CAFÉ BELLISSIMO INTENSO 200GR","CAFÉ DELTA RITUAL MU 220G","CAFÉ SOLÚVEL DELTA FRASCO 200G","CAFÉ SOLÚVEL DELTA FRASCO 100G","CEVADA SOLÚVEL DELTA FRASCO 200G","BEBIDA CEREAIS DELTA C/20%CAFE FR 200G"]
#lojas=["CNT AMADORA","CNT BRAGA","CNT CASCAIS","CNT COIMBRASHOPPING","CNT MATOSINHOS","CNT LEIRIA","CNT SANTAREM","CNT VIANA","MDL T. VEDRAS","CNT PORTIMÃO"]

In [7]:
# Definir o dataframe final apenas tendo em conta as lojas e os produtos requeridos

dfFinal = dfTudo[dfTudo['STORE_NAME'].isin(Lojas) & dfTudo['DESC_ARTIGO'].isin(produtos)].copy()

# Temos
- dfFinal com as métricas todas

# Queremos
- Reposição
- Dias específicos
- Colunas de dia e semana
- ID

# Fins de Semana e Semana

- função separar fins de semana

In [10]:
#Função para definir qual fim de semana queremos pedir
def fds(nome, df, numero):
    a = int(numero)
    b = 3*(a-1)
    nome=df[b:b+3].copy()
    return nome


> - Extrair apenas os fins de semana

In [11]:
dfFds = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])].copy()         #Sexta, Sábado e Domingo
#dfSemana = dfJuntos[dfJuntos['DATA'].dt.weekday.isin([0,1,2,3])]    #Segunda a Quinta

> - Fins de semana homólogos

In [12]:
#Dias em causa (1 fds depois da páscoa)
dfFdsPiloto=dfFds.loc[(dfFds['DATA'] >= '2022-04-22') & (dfFds['DATA'] <= '2022-05-15')].copy()

#Organizar para dar para ler mais facilmente
dfFdsPiloto=dfFdsPiloto.sort_values(by=["STORE", "DESC_ARTIGO","DATA"])

> - Colunas para a **semana** e o **dia**

In [13]:
# Copiar para fazer alterações mantendo o original
dfFdsFinal = dfFdsPiloto.copy()

#Semana
dfFdsFinal['Semana'] = dfFdsFinal['DATA'].dt.isocalendar().week
dfFdsFinal['Semana'] = dfFdsFinal.groupby('Semana').ngroup() + 1

#Dia
nome_dia = dfFdsFinal['DATA'].dt.day_name().map({'Friday': 'Sexta', 'Saturday': 'Sábado', 'Sunday': 'Domingo'})
dfFdsFinal['Dia'] = nome_dia

 - Criar um **índice** para cada dia

In [14]:
df=dfFdsFinal.copy()

# Extract the day and month components from the 'datetime' column
df['day'] = pd.to_datetime(df['DATA']).dt.day
df['month'] = pd.to_datetime(df['DATA']).dt.month

# Create a new column combining day and month
df['day_month'] = df['day'].astype(str) + '-' + df['month'].astype(str)

# Assign unique IDs to each unique day-month combination
df['id'], _ = pd.factorize(df['day_month'])

df = df.drop(columns=['day', 'month', 'day_month'])

# Temos:
- Uma base apenas com os fins de semana em causa com todas as métricas que estavam até agora mais a **Semana**, o **Dia** e o **Id** para poder ser mergido.

# Escrever

In [48]:
escrever_excel(df, "2_Delta_2022Id")

# Média de Stock por loja e produto

In [19]:
file1MedSto = dfFdsPiloto.groupby(['STORE_NAME', 'DESC_ARTIGO'])['SOH'].mean().reset_index()
file1StdSto = dfFdsPiloto.groupby(['STORE_NAME', 'DESC_ARTIGO'])['SOH'].std().reset_index()

stocks1 = pd.merge(file1MedSto, file1StdSto[['STORE_NAME', 'DESC_ARTIGO', 'SOH']], on=['STORE_NAME', 'DESC_ARTIGO'])
stocks1 = stocks1.rename(columns={'SOH_x': 'Stock_médio_Fds', 'SOH_y': 'sdSto'})